In [ ]:
# loading dependencies and plotting options
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import rioxarray as rxr
from scipy.interpolate import interp1d
from matplotlib.dates import DateFormatter

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

## SET USER DEFINITIONS

In [ ]:
DOMID = 'DAN'

# specify the year the reconstruction is being processed
year_value = 2023
# provided the DSD data blocks from 3_process_DSD.ipynb, what index does this year correspond to?
year_idx = 4

meadows_direc = '/Users/jpflug/Documents/Projects/cubesatReanaly/Data/Meadows/'+DOMID+'/self_classified/SCA/'
# dsd file for this year
dsd_file = meadows_direc+'processed_'+str(year_value)+'/DSD.npy'
# time series of snow cover and correpdonding dates
dsd_cubeFile = meadows_direc+'processed_'+str(year_value)+'/DSD_stacked.npy'
dsd_daydiffFile = meadows_direc+'processed_'+str(year_value)+'/DSD_daydiff.npy'
# 4 snow classes (see 4_process_SnowClass.ipynb)
class_file = meadows_direc+'classified_quartile.npy'
# base directory for the snow pillow data
pillow_direc = '/Users/jpflug/Documents/Projects/cubesatReanaly/Data/PillowData/'+DOMID+'/'

#### Load data from previous processing

In [ ]:
# load the date of snow disappearance for the reconstruction year
dsd = np.load(dsd_file)[year_idx,:,:]
# plot DSD for a sanity check
fg,ax = plt.subplots(figsize=(10,10))
out = ax.imshow(dsd,interpolation='none')
fg.colorbar(out)

# load the snow cover map data cube and corresponding dates (see 3_process_DSD)
dsdCube = np.load(dsd_cubeFile)
daydiff = np.load(dsd_daydiffFile)
# load the 4 unique snow classes (see 4_process_SnowClass)
snow_classes = np.load(class_file)
# plot as a sanity check
fg,ax = plt.subplots(figsize=(10,10))
ax.imshow(snow_classes)
# process the class ID's to loop through later in the script
class_numbers = np.unique(snow_classes)
class_numbers = class_numbers[~np.isnan(class_numbers)]

# load the snow pillow melt factor (MF), snowmelt onset date (MD), and shortwave radiation (dailySW)
start_wy = pd.Timestamp(str(year_value-1)+'-10-01 00:00:00')
MF = np.load(pillow_direc+str(year_value)+'/MF.npy')
MD = np.load(pillow_direc+str(year_value)+'/MD.npy',allow_pickle=True).item()
MD = (MD-start_wy).days
dailySW = np.load(pillow_direc+str(year_value)+'/meltSeason_SW.npy')

#### Process the full domain fSCA evolution

In [ ]:
# mask the stacked snow binary presence/absence map
classes_stacked = np.tile(snow_classes,[len(daydiff),1,1])
dsdCube[np.isnan(classes_stacked)] = np.nan
# classify the total modeling area based on the snow class map
total_area = len(snow_classes[~np.isnan(snow_classes)])
# calculate the fsca for the full domain
fsca = np.nansum(dsdCube,axis=(1,2))/total_area

# reduce the period to avoid overweighting on zero snow dates for reconstruction Methods 2 and 3
idx_start = 0
idx_end = np.where(fsca <= 0.01)
if len(idx_end) >= 1:
    try:
        idx_end = idx_end[0][0]+10
    except:
        idx_end = idx_end[0][0]+1
else:
    idx_end = len(fsca)+1
daydiff_filt = daydiff[idx_start:idx_end]
obs_date_list = [datetime.datetime(year_value-1, 10, 1) + datetime.timedelta(days=d) for d in daydiff_filt.astype(list)]
fsca_filt = fsca[idx_start:idx_end]
dsdCube_filt = dsdCube[idx_start:idx_end,:,:]
dsd_filt = dsd.copy()
# filter the dsd map by only grid cells falling within the unique snow classes
dsd_filt[np.isnan(snow_classes)] = np.nan

# plot the full-domain fsca versus time
fg,ax = plt.subplots()
ax.scatter(daydiff,fsca)
ax.scatter(daydiff[idx_start:idx_end],fsca[idx_start:idx_end],10,'r')
ax.plot([daydiff[idx_start],daydiff[idx_start]],[0,1],'--k')
# for reference, plot the snow pillow oberved date of snowmelt onset (MD)
ax.plot([MD,MD],[0,1],'--k')

#### Using reconstruction Method 1, estimate SWE on the initial spring date

In [ ]:
# determine the unique PS-observed snow disappearance dates
dsd_unique = np.unique(dsd)
dsd_unique = dsd_unique[~np.isnan(dsd_unique)]

# initialize
SWE_map_base = np.empty(dsd.shape)
SWE_map_base[:] = np.nan
# loop through the snow disappearance dates
for dsd_sel in dsd_unique:
    # check to see if the dsd date (daydiff_filt[0]) is before the modeling date, if so than the SWE disappears on that date
    if dsd_sel < daydiff_filt[0]:
        SWE_map_base[dsd_filt == dsd_sel] = 0
    else:
        # determine the cumulative radiation over this period
        swsum = dailySW.copy()
        swsum[:daydiff_filt[0]] = 0
        swsum = np.cumsum(swsum)[int(dsd_sel)]
        # calculate the SWE based on the pillow-defind MF
        SWE_map_base[dsd_filt == dsd_sel] = (swsum*MF[0])+MF[1]

# plot the Method 1 SWE estiamte on the initial date (t = daydiff_filt[0])
fg,ax = plt.subplots(figsize=(10,10))
out = ax.imshow(SWE_map_base,vmin=0,vmax=4.5,cmap='Blues')
fg.colorbar(out)
ax.set_facecolor([0.6,0.6,0.6])

#### Method 2: Assume uniform melt, but fit the melt rate using an assumption of meadow-scale heterogeneity

In [ ]:
############## FUNCTIONS -- DO NOT CHANGE ################
# lognormal distribution generator given mean SWE and CV
def get_SWEdist(D,meanSWE,CV):
    zeta = np.sqrt(np.log(1 + (CV**2)))
    lam = np.log(meanSWE) - (0.5*(zeta ** 2))
    fD = (1/(D*zeta*np.sqrt(2*np.pi)))*np.exp(-0.5*((np.log(D)-lam)/zeta)**2)
    fD = fD/np.sum(fD)
    return fD
# approach from Liston (2004) that calculates fSCA from portion of distribution higher than melt    
def listonApproach(D,meanSWE,CV,SW,MD,MF,offset):    
    # lognormal distribution generator given mean SWE and CV
    fD = get_SWEdist(D,meanSWE,CV)
    # calculate the cumulative shortwave radiation
    SW[:MD] = 0
    SW = np.cumsum(SW)
    # initialize and calculate the melt
    fsca_out = []
    # loop through the time record of shortwave data
    for sw_sel in SW:
        # calculate melt and shift the distribution uniformly
        melt = (sw_sel * MF) + offset
        newSWE = D - melt
        # calculate fSCA as the portion of the shifted distribution that is above zero
        fsca_out.append(np.sum(fD[newSWE > 0]))
    fsca_out = np.array(fsca_out)
    return fsca_out
############ END FUNCTIONS FOR METHOD 2 #################

## USER DEFINITIONS ##
# possible range of SWE values
D = np.linspace(0.0001,7,2000)
# range of SWE CoVs
C_array = np.linspace(0.05,0.5,20)
# range of possible melt factors
B_array = np.linspace(MF[0]*0.5,MF[0]*1.5,20)

# initialize
MSE = []
C_saver,B_saver,offset_saver = [],[],[]

fg,ax = plt.subplots()
# loop through the coefficients of variation
for C in C_array:
    # calculate the initial distribution based on the CoV and mean SWE
    temp_dist = get_SWEdist(D,np.nanmean(SWE_map_base),C)
    # allow the initial distribution to be shifted by +- 10%, loop through these shifts
    offsets = np.linspace(-0.1*np.nanmean(SWE_map_base),0.1*np.nanmean(SWE_map_base),20)
    for offset in offsets:
        # loop through the melt factors
        for B in B_array:
            # append each combination of model parameters into arrays
            C_saver.append(C)
            B_saver.append(B)
            offset_saver.append(offset)
            # fSCA output for this combination of model parameters (CVs, offsets, and melt factors)
            test_output = listonApproach(D,np.nanmean(SWE_map_base),C,dailySW,daydiff_filt[0],B,offset) 
            # plot a gray trace for each ensemble member's fSCA evolution
            ax.plot(test_output,color=[0.75,0.75,0.75])
            # calculate model performance and save -- using mean squared error
            MSE.append(np.mean((test_output[daydiff_filt]-fsca_filt)**2))

# plot the planet-observed fSCA evolution
ax.scatter(daydiff_filt,fsca_filt,20,'k',zorder=100)
# find best performing ensemble member and corresponding model parameters
best_idx = np.where(MSE == np.nanmin(MSE))[0][0]
B_single = B_saver[best_idx]
C_single = C_saver[best_idx]
offset_single = offset_saver[best_idx]
print('best model parameters')
print('Melt Factor: 'B_single)
print('SWE CoV: ',C_single)
print('Initial SWE offset: ',offset_single)

# re-generate the fSCA evolution using the best SWE parameters for Method 2, plot for comparison versus PS fSCA observations
test_output = listonApproach(D,np.nanmean(SWE_map_base),C_single,dailySW,daydiff_filt[0],B_single,offset_single) 
ax.plot(test_output,'--k')

### save the data, if desired ######
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method2_MeltFactor.npy',B_single)
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method2_CoV.npy',C_single)
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method2_offset.npy',offset_single)
#####################################

#### Method 3: Assume uniform melt, but fit the melt rate using an assumption of SWE heterogeneity in four snow classes 

In [ ]:
# initialize
B_class,C_class,offset_class = [],[],[]
# loop through the specific class numbers, 
# repeat the approach for Method 2 above, but for the unique classes
for class_no in class_numbers:
    # initialize, subclass
    MSE = []
    C_saver,B_saver,offset_saver = [],[],[]
    
    # mask the stacked snow binary presence/absence map based on the snow class of focus
    classes_stacked = np.tile(snow_classes,[len(daydiff),1,1])
    dsdCube_classes = dsdCube.copy()
    dsdCube_classes[classes_stacked != class_no] = np.nan
    # calculate fsca evolution for the specific snow class
    total_area = len(snow_classes[snow_classes == class_no])
    fsca_class = np.nansum(dsdCube_classes,axis=(1,2))/total_area
    fsca_class = fsca_class[idx_start:idx_end]
    
    fg,ax = plt.subplots()
    # loop through the SWE coefficients of variation
    for C in C_array:
        # calculate the initial distribution based on the CoV and mean SWE estimate of the specific class
        temp_dist = get_SWEdist(D,np.nanmean(SWE_map_base[snow_classes == class_no]),C)
        # allow the initial distribution to vary by +- 10%, loop through those shifts
        offsets = np.linspace(-0.1*np.nanmean(SWE_map_base),0.1*np.nanmean(SWE_map_base),20)
        for offset in offsets:
            # loop through the melt factors
            for B in B_array:
                # append into the subclass arrays
                C_saver.append(C)
                B_saver.append(B)
                offset_saver.append(offset)
                
                # fSCA evolution for this combination of CV and melt factors
                test_output = listonApproach(D,np.nanmean(SWE_map_base[snow_classes == class_no]),C,dailySW,daydiff_filt[0],B,offset) 
                ax.plot(test_output,color=[0.75,0.75,0.75])
                # save the stats
                MSE.append(np.mean((test_output[daydiff_filt]-fsca_class)**2))

    # calculate the best-performing model parameters for this snow class
    best_idx = np.where(MSE == np.nanmin(MSE))[0][0]
    B_class.append(B_saver[best_idx])
    C_class.append(C_saver[best_idx])
    offset_class.append(offset_saver[best_idx])

    # re-calculate the class-specific fSCA evolution from the best-performing parameters and plot
    test_output = listonApproach(D,np.nanmean(SWE_map_base[snow_classes == class_no]),
                                 C_class[-1],dailySW,daydiff_filt[0],B_class[-1],offset_class[-1]) 
    ax.plot(test_output,'--r')
    # plot both the domain-scale (black) and class-specific (red) fSCA observed by Planet
    ax.scatter(daydiff_filt,fsca_filt,20,'k',zorder=100)
    ax.scatter(daydiff_filt,fsca_class,20,'r',zorder=200)

### save the data, if desired ######
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method3_MeltFactor.npy',B_class)
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method3_CoV.npy',C_class)
# np.save(meadows_direc+'processed_'+str(year_value)+'/Method3_offset.npy',offset_class)
##################################

In [ ]:
# plot the initial estimated SWE distributions (top) and resulting fSCA evolution from Methods 2 and 3 (bottom)
fg,ax = plt.subplots(2,1,figsize=(7.5,6.2))

# specify colormap for the unique snow classes (see 4_process_SnowClass)
cols = ['#000000','#9E5546','#E5782E','#F9C71F']

# load parameters
B_class = np.load(meadows_direc+'processed_'+str(year_value)+'/Method3_MeltFactor.npy')
C_class = np.load(meadows_direc+'processed_'+str(year_value)+'/Method3_CoV.npy')
offset_class = np.load(meadows_direc+'processed_'+str(year_value)+'/Method3_offset.npy')
B_single = np.load(meadows_direc+'processed_'+str(year_value)+'/Method2_MeltFactor.npy')
C_single = np.load(meadows_direc+'processed_'+str(year_value)+'/Method2_CoV.npy')
offset_single = np.load(meadows_direc+'processed_'+str(year_value)+'/Method2_offset.npy')

# initialize
dist_holder = np.empty((class_numbers.shape[0],D.shape[0]))
dist_holder[:] = np.nan
melt_array = np.empty((class_numbers.shape[0],swsum.shape[0]))
melt_array[:] = np.nan
depth_array = np.empty((class_numbers.shape[0],D.shape[0]))
depth_array[:] = np.nan

# calculate the running cumulative shortwave radiation and melt array
swsum = dailySW.copy()
swsum[:daydiff_filt[0]] = 0
swsum = np.cumsum(swsum)

# loop through each class, determine the best SWE distribution and corresponding melt
for classCount,classNo in enumerate(class_numbers):
    class_meanSWE = np.mean(SWE_map_base[snow_classes == classNo])
    temp = get_SWEdist(D,class_meanSWE,C_class[classCount])
    temp = temp * (len(snow_classes[snow_classes == classNo])/len(snow_classes[~np.isnan(snow_classes)]))
    dist_holder[classCount,:] = temp        
    melt_array[classCount,:] = (swsum*B_class[classCount])     
    depth_array[classCount,:] = D-offset_class[classCount]

# interpolate the melt and resulting depth to a common benchmark (D)
depth_array_corrected = depth_array.copy()
dist_holder_corrected = dist_holder.copy()
for classCount,classNo in enumerate(class_numbers):
    depth_array_corrected[classCount,:] = D
    fx = interp1d(depth_array[classCount,:],dist_holder[classCount,:],fill_value='extrapolate')
    dist_holder_corrected[classCount,:] = fx(D)

# plot the class-specific SWE distributions
dist_holder_corrected = dist_holder_corrected/np.sum(dist_holder_corrected)
for classCount,classNo in enumerate(class_numbers):
    ax[0].plot(D,dist_holder_corrected[classCount,:],
               color=cols[classCount],linewidth=2,label='_nolegend_')
# inputting for the legend formatting        
ax[0].plot([np.nan,np.nan],[np.nan,np.nan],color='k',linewidth=2,label='Method 3')

# calculate the SWE distribution and resulting fSCA for the domain-scale method (Method 2)
SWE_dist_single = get_SWEdist(D,np.nanmean(SWE_map_base),C_single)
fx = interp1d(D-offset_single,SWE_dist_single,fill_value='extrapolate')
SWE_dist_single = fx(D)
ax[0].plot(D,SWE_dist_single,'--k',label='Method 2')
fsca_variable = []
for dtt in np.arange(melt_array.shape[1]):
    # print(dtt)
    temp = depth_array_corrected - np.tile(melt_array[:,dtt],[D.shape[0],1]).T
    fsca_variable.append(np.sum(dist_holder_corrected[temp > 0])) 
fsca_variable = np.array(fsca_variable)
test = listonApproach(D,np.nanmean(SWE_map_base),C_single,dailySW,daydiff_filt[0],B_single,offset_single) 
timeline = np.arange(len(test)).astype(float)
fsca_variable[timeline < daydiff_filt[0]] = np.nan
fsca_variable[timeline > np.max(daydiff_filt)] = np.nan
test[timeline < daydiff_filt[0]] = np.nan
test[timeline > np.max(daydiff_filt)] = np.nan
# specify the dates to plot agains
date_list = [datetime.datetime(year_value-1, 10, 1) + datetime.timedelta(days=d) for d in timeline]

# plot the domain-scale fSCA evolution for both Method 2 and Method 3
ax[1].plot(date_list,fsca_variable,'-k',label='Method 3')
ax[1].plot(date_list,test,'--k',label='Method 2')

# plot the PlanetScope fSCA observations for comparison
obs_date_list = [datetime.datetime(year_value-1, 10, 1) + datetime.timedelta(days=d) for d in daydiff_filt.astype(list)]
ax[1].scatter(obs_date_list,fsca_filt,20,'k',zorder=100,label='Planet observations')

# plot formatting
ax[0].set_xlim([0,2.5])
ax[0].set_xlabel('Snow Water Equivalent [m]')
ax[0].set_ylabel('PDF')
ax[0].legend()
ax[0].set_title('Station ID: '+DOMID+', Water Year: '+str(year_value))
ax[1].set_ylabel('fractional\nsnow-covered area')
ax[1].legend()
date_formatter = DateFormatter('%b %d')
ax[1].xaxis.set_major_formatter(date_formatter)